In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install hyperimpute

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 27.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11

In [ ]:
from hyperimpute.plugins.imputers import Imputers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_data=pd.read_csv("/content/drive/MyDrive/data_timeseries_minmaxscaler.csv")

In [ ]:
#preprocess
ids = {'50882': 'Bic', '50912': 'Crt', '50971': 'Pot', '50983': 'Sod', '51006': 'Ure', '51222': 'Hgb', '51265': 'Plt', '51301': 'Wbc'}
df=df_data.rename(columns=ids)
df=df.dropna()

In [ ]:
#get matrix for each row in dataframe
columns_to_scale = ['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']
df['matrix']=df[columns_to_scale].apply(lambda row: [row[col].strip('][').split(', ') for col in columns_to_scale],axis=1)

In [ ]:
def get_dfs(data):
  dataframes=[]
  for matrix in data:
    df=pd.DataFrame(matrix).transpose()
    df.columns=['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']
    dataframes.append(df)
  return dataframes

In [ ]:
dataframes=get_dfs(df['matrix'].values)

In [ ]:
padding = pd.DataFrame(np.zeros((4 - len(out), len(out.columns))), columns=out.columns)
mini_df = pd.concat([out, padding])

In [ ]:
dataframes[11]

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,0.2153846153846154,0.005405405405405406,0.10546875,0.5412844036697249,0.013114754098360656,0.5150214592274678,0.052552552552552555,0.013890470226574064
1,0.2153846153846154,0.005405405405405406,0.09375,0.559633027522936,0.01639344262295082,0.5493562231759657,0.06906906906906907,0.013207332346578619
2,0.19230769230769232,0.004054054054054054,0.08984375,0.5321100917431194,0.013114754098360656,0.5236051502145922,0.07695195195195195,0.013093476033246045
3,0.17692307692307693,0.004054054054054054,0.10546875,0.5137614678899083,0.01639344262295082,0.5150214592274678,0.07357357357357357,0.013890470226574064
4,0.19230769230769232,0.004054054054054054,0.1328125,0.5137614678899083,0.01639344262295082,0.5407725321888411,0.08596096096096097,0.01400432653990664
5,0.18461538461538463,0.005405405405405406,0.1328125,0.5504587155963303,0.013114754098360656,0.4849785407725322,0.09684684684684684,0.01377661391324149


In [ ]:
d=dataframes[0]
p=d.copy()
mask = np.random.choice([True, False], size=d.shape, p=[0.4, 0.6])
d[mask] = np.nan

In [ ]:
if len(d) < 4:
    padding = pd.DataFrame(np.zeros((4 - len(d), len(d.columns))), columns=d.columns)
    mini_df = pd.concat([d, padding])

In [ ]:
from tqdm import tqdm

In [ ]:
dataframes[0]

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,0.2,0.004054054054054054,0.140625,0.5871559633027523,0.08196721311475409,0.5450643776824035,0.024774774774774775,0.004781965159968121


In [ ]:
modified_mini_dfs = []
ids = {0: 'Bic', 1: 'Crt', 2: 'Pot', 3: 'Sod', 4: 'Ure', 5: 'Hgb', 6: 'Plt', 7: 'Wbc'}
for mini_df in dataframes:
    # If the mini dataframe has more than 4 rows, use only the first 4 rows
    if len(mini_df) > 4:
        mini_df = mini_df.head(4)
    # Randomly mask 40% of the values as NaN
    preMask=mini_df.copy()
    mask = np.random.choice([True, False], size=mini_df.shape, p=[0.4, 0.6])
    mini_df[mask] = np.nan

    plugin = Imputers().get('mice')
    out = plugin.fit_transform(mini_df.copy())

    if len(out.columns)!=8: #not enough values to impute
      # If the mini dataframe has less than 4 rows, pad it with zeroes
      if len(mini_df) < 4:
        padding = pd.DataFrame(np.zeros((4 - len(preMask), len(preMask.columns))), columns=preMask.columns)
        mini_df = pd.concat([preMask, padding])
      modified_mini_dfs.append(mini_df)
      continue

    # If the mini dataframe has less than 4 rows, pad it with zeroes
    if len(out) < 4:
        padding = pd.DataFrame(np.zeros((4 - len(out), len(out.columns))), columns=out.columns)
        out = pd.concat([out, padding])

    # Append the inputed mini dataframe to the list
    modified_mini_dfs.append(out)

Streaming output truncated to the last 5000 lines.
  mini_df[mask] = np.nan
<ipython-input-167-5d60b20541f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_df[mask] = np.nan
<ipython-input-167-5d60b20541f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_df[mask] = np.nan
<ipython-input-167-5d60b20541f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_df[mas

KeyboardInterrupt: 

In [ ]:
modified_mini_dfs[54]

,0,1,2,3,4,5,6,7
0,0.200000,0.012162,0.113281,0.605505,0.072131,0.553648,0.069069,0.005124
1,0.207692,0.012162,0.101562,0.596330,0.049180,0.540773,0.069069,0.005124
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
import pickle

In [ ]:
with open("/content/drive/MyDrive/safeguardMICE_df.pkl", "wb") as fp:   #Pickling
  pickle.dump(modified_mini_dfs, fp)

In [ ]:
with open("/content/drive/MyDrive/safeguard_df.pkl", "rb") as fp:   #unPickling
  readingTest=pickle.load(fp)

In [ ]:
mini_df=modified_mini_dfs[10]
[mini_df[col].tolist() for col in mini_df.columns]

[[0.14615384615384616,
  0.17692307692307693,
  0.16153846153846155,
  0.16153846153846155],
 [0.02027027027027027,
  0.02072072072072072,
  0.02027027027027027,
  0.021621621621621623],
 [0.12890625, 0.11328125, 0.11328125, 0.11848958333333333],
 [0.6422018348623855, 0.63302752293578, 0.6146788990825689, 0.63302752293578],
 [0.10819672131147541,
  0.09836065573770492,
  0.10163934426229508,
  0.10491803278688525],
 [0.6137339055793992,
  0.5450643776824035,
  0.51931330472103,
  0.5236051502145922],
 [0.06156156156156156,
  0.05198948948948949,
  0.04241741741741742,
  0.05198948948948949],
 [0.011841056586587727,
  0.007059091426619608,
  0.005123534099965844,
  0.00421268359330525]]

In [ ]:
columns_to_scale = ['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']

In [ ]:
modified_mini_dfs[0]['Bic'].values.astype(float)

array([0.2, 0. , 0. , 0. ])

In [ ]:
imputed_df.iloc[0][columns_to_scale[0]]

'[0.2]'

In [ ]:
ids = {'0': 'Bic', '1': 'Crt', '2': 'Pot', '3': 'Sod', '4': 'Ure', '5': 'Hgb', '6': 'Plt', '7': 'Wbc'}
modified_mini_dfs[10].rename(columns=ids)

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,0.146154,0.020270,0.128906,0.642202,0.108197,0.613734,0.061562,0.011841
1,0.176923,0.020721,0.113281,0.633028,0.098361,0.545064,0.051989,0.007059
2,0.161538,0.020270,0.113281,0.614679,0.101639,0.519313,0.042417,0.005124
3,0.161538,0.021622,0.118490,0.633028,0.104918,0.523605,0.051989,0.004213


In [ ]:
modified_mini_dfs[10][0]

0    0.146154
1    0.176923
2    0.161538
3    0.161538
Name: 0, dtype: float64

In [ ]:
df.loc[0,columns_to_scale[0]]

'[0.2]'

In [ ]:
summary_list=[]
for pi_df in modified_mini_dfs:
  pi_vals_list=[f'{pi_df[col].values.astype(float).tolist()}' for col in pi_df.columns]
  summary_list.append(pi_vals_list)

In [ ]:
summary_list[:5]

[['[0.2, 0.0, 0.0, 0.0]',
  '[0.004054054054054054, 0.0, 0.0, 0.0]',
  '[0.140625, 0.0, 0.0, 0.0]',
  '[0.5871559633027523, 0.0, 0.0, 0.0]',
  '[0.08196721311475409, 0.0, 0.0, 0.0]',
  '[0.5450643776824035, 0.0, 0.0, 0.0]',
  '[0.024774774774774775, 0.0, 0.0, 0.0]',
  '[0.004781965159968121, 0.0, 0.0, 0.0]'],
 ['[0.17692307692307693, 0.0, 0.0, 0.0]',
  '[0.004054054054054054, 0.0, 0.0, 0.0]',
  '[0.16796875, 0.0, 0.0, 0.0]',
  '[0.48623853211009194, 0.0, 0.0, 0.0]',
  '[0.09508196721311475, 0.0, 0.0, 0.0]',
  '[0.5321888412017167, 0.0, 0.0, 0.0]',
  '[0.04954954954954955, 0.0, 0.0, 0.0]',
  '[0.007514516679949903, 0.0, 0.0, 0.0]'],
 ['[0.18461538461538463, 0.16923076923076924, 0.0, 0.0]',
  '[0.008108108108108109, 0.008108108108108109, 0.0, 0.0]',
  '[0.2109375, 0.17578125, 0.0, 0.0]',
  '[0.48623853211009194, 0.45871559633027537, 0.0, 0.0]',
  '[0.12131147540983607, 0.09508196721311475, 0.0, 0.0]',
  '[0.51931330472103, 0.49785407725321884, 0.0, 0.0]',
  '[0.04804804804804805, 0.04804

In [ ]:
summary_df=pd.DataFrame(summary_list, columns=columns_to_scale)
summary_df

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,"[0.2, 0.0, 0.0, 0.0]","[0.004054054054054054, 0.0, 0.0, 0.0]","[0.140625, 0.0, 0.0, 0.0]","[0.5871559633027523, 0.0, 0.0, 0.0]","[0.08196721311475409, 0.0, 0.0, 0.0]","[0.5450643776824035, 0.0, 0.0, 0.0]","[0.024774774774774775, 0.0, 0.0, 0.0]","[0.004781965159968121, 0.0, 0.0, 0.0]"
1,"[0.17692307692307693, 0.0, 0.0, 0.0]","[0.004054054054054054, 0.0, 0.0, 0.0]","[0.16796875, 0.0, 0.0, 0.0]","[0.48623853211009194, 0.0, 0.0, 0.0]","[0.09508196721311475, 0.0, 0.0, 0.0]","[0.5321888412017167, 0.0, 0.0, 0.0]","[0.04954954954954955, 0.0, 0.0, 0.0]","[0.007514516679949903, 0.0, 0.0, 0.0]"
2,"[0.18461538461538463, 0.16923076923076924, 0.0...","[0.008108108108108109, 0.008108108108108109, 0...","[0.2109375, 0.17578125, 0.0, 0.0]","[0.48623853211009194, 0.45871559633027537, 0.0...","[0.12131147540983607, 0.09508196721311475, 0.0...","[0.51931330472103, 0.49785407725321884, 0.0, 0.0]","[0.04804804804804805, 0.04804804804804805, 0.0...","[0.008539223499943073, 0.008539223499943073, 0..."
3,"[0.16923076923076924, 0.19230769230769232, 0.0...","[0.005405405405405406, 0.005405405405405406, 0...","[0.16015625, 0.16796875, 0.0, 0.0]","[0.5229357798165137, 0.5229357798165137, 0.0, ...","[0.09180327868852459, 0.11475409836065574, 0.0...","[0.5107296137339056, 0.4806866952789699, 0.0, ...","[0.03340840840840841, 0.03340840840840841, 0.0...","[0.004668108846635546, 0.004668108846635546, 0..."
4,"[0.13076923076923078, 0.16923076923076924, 0.1...","[0.010810810810810811, 0.009909909909909911, 0...","[0.19140625, 0.12109375, 0.125, 0.145833333333...","[0.5412844036697249, 0.559633027522936, 0.5871...","[0.05573770491803279, 0.036065573770491806, 0....","[0.5493562231759657, 0.5708154506437768, 0.596...","[0.09684684684684684, 0.10266516516516516, 0.1...","[0.009791642946601389, 0.009450074006603667, 0..."
...,...,...,...,...,...,...,...,...
311055,"[0.1076923076923077, 0.13846153846153847, 0.12...","[0.00945945945945946, 0.007657657657657658, 0....","[0.12109375, 0.109375, 0.08984375, 0.106770833...","[0.5963302752293578, 0.5871559633027523, 0.614...","[0.04262295081967213, 0.022950819672131147, 0....","[0.49356223175965663, 0.45922746781115875, 0.3...","[0.13626126126126126, 0.128003003003003, 0.119...","[0.014915177046567233, 0.012676002884359938, 0..."
311056,"[0.16923076923076924, 0.16153846153846155, 0.1...","[0.00945945945945946, 0.007657657657657658, 0....","[0.1484375, 0.1328125, 0.12109375, 0.134114583...","[0.5779816513761469, 0.5963302752293578, 0.633...","[0.036065573770491806, 0.01639344262295082, 0....","[0.424892703862661, 0.41630901287553645, 0.386...","[0.18843843843843844, 0.15878378378378377, 0.1...","[0.010816349766594558, 0.009108505066605944, 0..."
311057,"[0.2076923076923077, 0.17692307692307693, 0.19...","[0.008108108108108109, 0.009909909909909911, 0...","[0.09375, 0.11328125, 0.11328125, 0.1067708333...","[0.5963302752293578, 0.6146788990825689, 0.587...","[0.03278688524590164, 0.06229508196721312, 0.0...","[0.6266094420600858, 0.6652360515021459, 0.643...","[0.10022522522522523, 0.1036036036036036, 0.10...","[0.020038711146533078, 0.020266423773198226, 0..."
311058,"[0.13846153846153847, 0.19230769230769232, 0.1...","[0.010810810810810811, 0.010810810810810811, 0...","[0.10546875, 0.1015625, 0.109375, 0.0]","[0.5963302752293578, 0.5963302752293578, 0.614...","[0.02622950819672131, 0.039344262295081964, 0....","[0.6266094420600858, 0.6351931330472104, 0.643...","[0.0762012012012012, 0.07657657657657657, 0.07...","[0.019924854833200504, 0.01724923146988501, 0...."


In [ ]:
df

,Unnamed: 0,subject_id,hadm_id,charttime,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc,matrix
0,0,10000032,22595853.0,['2180-05-07 05:05:00'],"[0.2, 0.0, 0.0, 0.0]","[0.004054054054054054, 0.0, 0.0, 0.0]","[0.140625, 0.0, 0.0, 0.0]","[0.5871559633027523, 0.0, 0.0, 0.0]","[0.08196721311475409, 0.0, 0.0, 0.0]","[0.5450643776824035, 0.0, 0.0, 0.0]","[0.024774774774774775, 0.0, 0.0, 0.0]","[0.004781965159968121, 0.0, 0.0, 0.0]","[[0.2], [0.004054054054054054], [0.140625], [0..."
1,1,10000032,22841357.0,['2180-06-27 05:10:00'],"[0.17692307692307693, 0.0, 0.0, 0.0]","[0.004054054054054054, 0.0, 0.0, 0.0]","[0.16796875, 0.0, 0.0, 0.0]","[0.48623853211009194, 0.0, 0.0, 0.0]","[0.09508196721311475, 0.0, 0.0, 0.0]","[0.5321888412017167, 0.0, 0.0, 0.0]","[0.04954954954954955, 0.0, 0.0, 0.0]","[0.007514516679949903, 0.0, 0.0, 0.0]","[[0.17692307692307693], [0.004054054054054054]..."
2,2,10000032,25742920.0,"['2180-08-06 06:36:00', '2180-08-07 06:15:00']","[0.18461538461538463, 0.16923076923076924, 0.0...","[0.008108108108108109, 0.008108108108108109, 0...","[0.2109375, 0.17578125, 0.0, 0.0]","[0.48623853211009194, 0.45871559633027537, 0.0...","[0.12131147540983607, 0.09508196721311475, 0.0...","[0.51931330472103, 0.49785407725321884, 0.0, 0.0]","[0.04804804804804805, 0.04804804804804805, 0.0...","[0.008539223499943073, 0.008539223499943073, 0...","[[0.18461538461538463, 0.16923076923076924], [..."
3,3,10000032,29079034.0,"['2180-07-24 06:35:00', '2180-07-25 04:45:00']","[0.16923076923076924, 0.19230769230769232, 0.0...","[0.005405405405405406, 0.005405405405405406, 0...","[0.16015625, 0.16796875, 0.0, 0.0]","[0.5229357798165137, 0.5229357798165137, 0.0, ...","[0.09180327868852459, 0.11475409836065574, 0.0...","[0.5107296137339056, 0.4806866952789699, 0.0, ...","[0.03340840840840841, 0.03340840840840841, 0.0...","[0.004668108846635546, 0.004668108846635546, 0...","[[0.16923076923076924, 0.19230769230769232], [..."
4,4,10000084,23052089.0,"['2160-11-20 22:30:00', '2160-11-21 07:00:00',...","[0.13076923076923078, 0.16923076923076924, 0.1...","[0.010810810810810811, 0.009909909909909911, 0...","[0.19140625, 0.12109375, 0.125, 0.145833333333...","[0.5412844036697249, 0.559633027522936, 0.5871...","[0.05573770491803279, 0.036065573770491806, 0....","[0.5493562231759657, 0.5708154506437768, 0.596...","[0.09684684684684684, 0.10266516516516516, 0.1...","[0.009791642946601389, 0.009450074006603667, 0...","[[0.13076923076923078, 0.16923076923076924, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311055,311055,19999828,25744818.0,"['2149-01-09 03:13:00', '2149-01-10 05:58:00',...","[0.1076923076923077, 0.13846153846153847, 0.16...","[0.00945945945945946, 0.008108108108108109, 0....","[0.12109375, 0.109375, 0.08984375, 0.1015625, ...","[0.5963302752293578, 0.5871559633027523, 0.614...","[0.04262295081967213, 0.022950819672131147, 0....","[0.49356223175965663, 0.45922746781115875, 0.3...","[0.13626126126126126, 0.12687687687687688, 0.1...","[0.014915177046567233, 0.013207332346578619, 0...","[[0.1076923076923077, 0.13846153846153847, 0.1..."
311056,311056,19999828,29734428.0,"['2147-07-17 18:04:00', '2147-07-19 04:53:00',...","[0.16923076923076924, 0.16153846153846155, 0.1...","[0.00945945945945946, 0.006756756756756757, 0....","[0.1484375, 0.1328125, 0.12109375, 0.10546875,...","[0.5779816513761469, 0.5963302752293578, 0.633...","[0.036065573770491806, 0.01639344262295082, 0....","[0.424892703862661, 0.41630901287553645, 0.386...","[0.18843843843843844, 0.17004504504504506, 0.1...","[0.010816349766594558, 0.008766936126608222, 0...","[[0.16923076923076924, 0.16153846153846155, 0...."
311057,311057,19999840,21033226.0,"['2164-09-11 05:50:00', '2164-09-12 06:45:00',...","[0.2076923076923077, 0.17692307692307693, 0.18...","[0.008108108108108109, 0.010810810810810811, 0...","[0.09375, 0.11328125, 0.11328125, 0.0859375, 0...","[0.5963302752293578, 0.6146788990825689, 0.587...","[0.03278688524590164, 0.06229508196721312, 0.0...","[0.6266094420600858, 0.6

In [ ]:
summary_df['hadm_id']=df['hadm_id']
summary_df['charttime']=df['charttime']
summary_df

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc,subject_id,hadm_id,charttime
0,"[0.2, 0.0, 0.0, 0.0]","[0.004054054054054054, 0.0, 0.0, 0.0]","[0.140625, 0.0, 0.0, 0.0]","[0.5871559633027523, 0.0, 0.0, 0.0]","[0.08196721311475409, 0.0, 0.0, 0.0]","[0.5450643776824035, 0.0, 0.0, 0.0]","[0.024774774774774775, 0.0, 0.0, 0.0]","[0.004781965159968121, 0.0, 0.0, 0.0]",10000032,22595853.0,['2180-05-07 05:05:00']
1,"[0.17692307692307693, 0.0, 0.0, 0.0]","[0.004054054054054054, 0.0, 0.0, 0.0]","[0.16796875, 0.0, 0.0, 0.0]","[0.48623853211009194, 0.0, 0.0, 0.0]","[0.09508196721311475, 0.0, 0.0, 0.0]","[0.5321888412017167, 0.0, 0.0, 0.0]","[0.04954954954954955, 0.0, 0.0, 0.0]","[0.007514516679949903, 0.0, 0.0, 0.0]",10000032,22841357.0,['2180-06-27 05:10:00']
2,"[0.18461538461538463, 0.16923076923076924, 0.0...","[0.008108108108108109, 0.008108108108108109, 0...","[0.2109375, 0.17578125, 0.0, 0.0]","[0.48623853211009194, 0.45871559633027537, 0.0...","[0.12131147540983607, 0.09508196721311475, 0.0...","[0.51931330472103, 0.49785407725321884, 0.0, 0.0]","[0.04804804804804805, 0.04804804804804805, 0.0...","[0.008539223499943073, 0.008539223499943073, 0...",10000032,25742920.0,"['2180-08-06 06:36:00', '2180-08-07 06:15:00']"
3,"[0.16923076923076924, 0.19230769230769232, 0.0...","[0.005405405405405406, 0.005405405405405406, 0...","[0.16015625, 0.16796875, 0.0, 0.0]","[0.5229357798165137, 0.5229357798165137, 0.0, ...","[0.09180327868852459, 0.11475409836065574, 0.0...","[0.5107296137339056, 0.4806866952789699, 0.0, ...","[0.03340840840840841, 0.03340840840840841, 0.0...","[0.004668108846635546, 0.004668108846635546, 0...",10000032,29079034.0,"['2180-07-24 06:35:00', '2180-07-25 04:45:00']"
4,"[0.13076923076923078, 0.16923076923076924, 0.1...","[0.010810810810810811, 0.009909909909909911, 0...","[0.19140625, 0.12109375, 0.125, 0.145833333333...","[0.5412844036697249, 0.559633027522936, 0.5871...","[0.05573770491803279, 0.036065573770491806, 0....","[0.5493562231759657, 0.5708154506437768, 0.596...","[0.09684684684684684, 0.10266516516516516, 0.1...","[0.009791642946601389, 0.009450074006603667, 0...",10000084,23052089.0,"['2160-11-20 22:30:00', '2160-11-21 07:00:00',..."
...,...,...,...,...,...,...,...,...,...,...,...
311055,"[0.1076923076923077, 0.13846153846153847, 0.12...","[0.00945945945945946, 0.007657657657657658, 0....","[0.12109375, 0.109375, 0.08984375, 0.106770833...","[0.5963302752293578, 0.5871559633027523, 0.614...","[0.04262295081967213, 0.022950819672131147, 0....","[0.49356223175965663, 0.45922746781115875, 0.3...","[0.13626126126126126, 0.128003003003003, 0.119...","[0.014915177046567233, 0.012676002884359938, 0...",19999828,25744818.0,"['2149-01-09 03:13:00', '2149-01-10 05:58:00',..."
311056,"[0.16923076923076924, 0.16153846153846155, 0.1...","[0.00945945945945946, 0.007657657657657658, 0....","[0.1484375, 0.1328125, 0.12109375, 0.134114583...","[0.5779816513761469, 0.5963302752293578, 0.633...","[0.036065573770491806, 0.01639344262295082, 0....","[0.424892703862661, 0.41630901287553645, 0.386...","[0.18843843843843844, 0.15878378378378377, 0.1...","[0.010816349766594558, 0.009108505066605944, 0...",19999828,29734428.0,"['2147-07-17 18:04:00', '2147-07-19 04:53:00',..."
311057,"[0.2076923076923077, 0.17692307692307693, 0.19...","[0.008108108108108109, 0.009909909909909911, 0...","[0.09375, 0.11328125, 0.11328125, 0.1067708333...","[0.5963302752293578, 0.6146788990825689, 0.587...","[0.03278688524590164, 0.06229508196721312, 0.0...","[0.6266094420600858, 0.6652360515021459, 0.643...","[0.10022522522522523, 0.1036036036036036, 0.10...","[0.020038711146533078, 0.020266423773198226, 0...",19999840,21033226.0,"['2164-09-11 05:50:00', '2164-09-12 06:45:00',..."
311058,"[0.13846153846153847, 0.19230769230769232, 0.1...","[0.010810810810810811, 0.010810810810810811, 0...","[0.10546875, 0.1015625, 0.109375, 0.0]","[0.5963302752293578, 0.5963302752293578, 0.614...","[0.02622950819672131, 0.039344262295081964, 0....","[0.6266094420600858, 0.6351931330

In [ ]:
summary_df.to_csv("/content/drive/MyDrive/meanImputedData.csv")